### Import necessary libraries

In [1]:
import findspark
from pyspark.sql import SparkSession
#Functions for Preprocessing
from functools import reduce
from pyspark.sql.functions import lower, col, sum, when, regexp_replace, trim
from pyspark.sql.types import StringType, LongType, TimestampType

In [3]:
findspark.init()

In [5]:
spark = SparkSession.builder \
    .appName("MongoDB Integration") \
    .config("spark.jars", "C:\\Users\\ADURA\\Documents\\spark_jars\\mongo-spark-connector_2.13-10.5.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\mongodb-driver-sync-4.9.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\mongodb-driver-core-4.9.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\bson-4.9.0.jar").getOrCreate()

In [7]:
df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
    .option("spark.mongodb.read.database", "tweet_db") \
    .option("spark.mongodb.read.collection", "tweets") \
    .load()

In [9]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- author_id: long (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)



In [11]:
df.show(5)

+--------------------+-------------------+--------------------+-------------------+--------------------+
|                 _id|          author_id|          created_at|                 id|                text|
+--------------------+-------------------+--------------------+-------------------+--------------------+
|6891a2e83a8bc8d49...| 986623019819954176|2025-08-04T13:42:...|1952364494056726579|@eurogamer Hopefu...|
|6891a2e83a8bc8d49...|1481237608063979522|2025-08-04T13:42:...|1952364487782334584|@CyberSaintHQ  La...|
|6891a2e83a8bc8d49...|          467061296|2025-08-04T13:42:...|1952364485701960138|RT @LeonWohlgemut...|
|6891a2e83a8bc8d49...|          257526240|2025-08-04T13:42:...|1952364483138957781|RT @RealPatrickWe...|
|6891a2e83a8bc8d49...|1168547410647355392|2025-08-04T13:42:...|1952364481600000265|I’ve been a DEX u...|
+--------------------+-------------------+--------------------+-------------------+--------------------+
only showing top 5 rows


In [12]:
print(spark.sparkContext._jsc.sc().listJars())

List(spark://DESKTOP-OBIQKOI:52457/jars/mongo-spark-connector_2.13-10.5.0.jar, spark://DESKTOP-OBIQKOI:52457/jars/mongodb-driver-core-4.9.0.jar, spark://DESKTOP-OBIQKOI:52457/jars/mongodb-driver-sync-4.9.0.jar, spark://DESKTOP-OBIQKOI:52457/jars/bson-4.9.0.jar)


In [13]:
df.dtypes

[('_id', 'string'),
 ('author_id', 'bigint'),
 ('created_at', 'string'),
 ('id', 'bigint'),
 ('text', 'string')]

### Data Cleaning

In [15]:
df = df.drop_duplicates()

In [17]:
df.distinct().count()

80

*__All 80 records are unique__*

In [20]:
missing_counts = df.select([sum(col(c).isNull().cast("int")).alias(c)
                            for c in df.columns])
missing_counts.show()

+---+---------+----------+---+----+
|_id|author_id|created_at| id|text|
+---+---------+----------+---+----+
|  0|        0|         0|  0|   0|
+---+---------+----------+---+----+



*__There are no missing values in any column__*

In [23]:
#Remove rows with irrelevant text
df = df.filter(df['text'] != '')

In [25]:
shape = (df.count(), len(df.columns))

In [26]:
shape

(80, 5)

In [29]:
#Text manipulation using regular expression (regex)
df = df.withColumn('text', regexp_replace(col('text'), r'http\S+', ''))  # Remove URLs
df = df.withColumn('text', regexp_replace(col('text'), r'@\w+', ''))     # Remove mentions
df = df.withColumn('text', regexp_replace(col('text'), r'#\w+', ''))     # Remove hashtags
df = df.withColumn('text', regexp_replace(col('text'), r'[^a-zA-Z\s]', ''))  # Remove special characters
df = df.withColumn('text', regexp_replace(col('text'), r'\n', ''))
df = df.withColumn('text', trim(col('text'))) #Remove leading or trailing white spaces

In [31]:
df.select('text').show(5)

+--------------------+
|                text|
+--------------------+
|OR COULD IT BE TH...|
|RT  Elon I worked...|
|In the US at leas...|
|RT  Disney planne...|
|RT  BREAKING Lock...|
+--------------------+
only showing top 5 rows


In [33]:
#Changing datatypes
df = df.withColumn('author_id', df['author_id'].cast(LongType()))
df = df.withColumn('created_at', df['created_at'].cast(TimestampType()))

In [35]:
df.dtypes

[('_id', 'string'),
 ('author_id', 'bigint'),
 ('created_at', 'timestamp'),
 ('id', 'bigint'),
 ('text', 'string')]

__*Long and bigint are similar*__

### Feature Engineering

In [37]:
# Preprocess text
df_clean = df.withColumn('text', lower(col('text')))
df_clean = df_clean.withColumn('text', col('text').cast("string"))

# Sentiment word lists
positive_words = ["good", "great", "excellent", "happy", "love", "amazing", "awesome"]
negative_words = ["bad", "terrible", "sad", "hate", "poor", "awful", "worst"]

In [39]:
df_clean.show(5)

+--------------------+-------------------+-------------------+-------------------+--------------------+
|                 _id|          author_id|         created_at|                 id|                text|
+--------------------+-------------------+-------------------+-------------------+--------------------+
|6891a2e83a8bc8d49...|1373906096792989696|2025-08-04 14:42:03|1952364410074234928|or could it be th...|
|6891a2e83a8bc8d49...|1756438119287566336|2025-08-04 14:41:53|1952364370069217638|rt  elon i worked...|
|6891a2e83a8bc8d49...|1154966639408013312|2025-08-04 14:42:06|1952364425240826229|in the us at leas...|
|6891a2e83a8bc8d49...|         3389907262|2025-08-04 14:42:20|1952364480869826560|rt  disney planne...|
|6891a2e83a8bc8d49...|1331232643330023429|2025-08-04 14:42:04|1952364414289764415|rt  breaking lock...|
+--------------------+-------------------+-------------------+-------------------+--------------------+
only showing top 5 rows


In [41]:
positive_condition = reduce(lambda a, b: a | b, [col("text").contains(word) for word in positive_words])
negative_condition = reduce(lambda a, b: a | b, [col("text").contains(word) for word in negative_words])

df_with_sentiment = df_clean.withColumn(
    "sentiment",
    when(positive_condition, "positive")
    .when(negative_condition, "negative")
    .otherwise("neutral")
)

In [43]:
df_with_sentiment.show(10)

+--------------------+-------------------+-------------------+-------------------+--------------------+---------+
|                 _id|          author_id|         created_at|                 id|                text|sentiment|
+--------------------+-------------------+-------------------+-------------------+--------------------+---------+
|6891a2e83a8bc8d49...|1373906096792989696|2025-08-04 14:42:03|1952364410074234928|or could it be th...|  neutral|
|6891a2e83a8bc8d49...|1756438119287566336|2025-08-04 14:41:53|1952364370069217638|rt  elon i worked...|  neutral|
|6891a2e83a8bc8d49...|1154966639408013312|2025-08-04 14:42:06|1952364425240826229|in the us at leas...|  neutral|
|6891a2e83a8bc8d49...|         3389907262|2025-08-04 14:42:20|1952364480869826560|rt  disney planne...|  neutral|
|6891a2e83a8bc8d49...|1331232643330023429|2025-08-04 14:42:04|1952364414289764415|rt  breaking lock...|  neutral|
|6891a2e83a8bc8d49...|          453639280|2025-08-04 14:41:59|1952364394152686018|whats 

In [45]:
df_with_sentiment.filter(col("text").like(f"%technology%")).count()

43

In [46]:
df_with_sentiment.filter(col("text").like(f"%climate%")).count()

13

In [49]:
df_with_sentiment.filter(col("sentiment").like(f"%neutral%")).count()

75

In [51]:
df_with_sentiment.filter(col("sentiment").like(f"%positive%")).count()

4

In [53]:
df_with_sentiment.filter(col("sentiment").like(f"%negative%")).count()

1

### Feature engineering continued - Using NLTK'S opinion lexicon for positive and negative words

In [55]:
def load_opinion_lexicon(path):
    with open(path, 'r', encoding='latin-1') as f:
        return [line.strip() for line in f if line.strip() and not line.startswith(';')]

positive_words2 = load_opinion_lexicon("C:\\Users\\ADURA\\Documents\\GitHub\\nltk\\nltk\\corpus\\positive-words.txt")
negative_words2 = load_opinion_lexicon("C:\\Users\\ADURA\\Documents\\GitHub\\nltk\\nltk\\corpus\\negative-words.txt")

In [57]:
positive_words2[:10]

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation']

In [59]:
type(positive_words2), type(negative_words2)

(list, list)

In [61]:
negative_words2[:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [63]:
positive_condition2 = reduce(lambda a, b: a | b, [col("text").contains(word) for word in positive_words2])
negative_condition2 = reduce(lambda a, b: a | b, [col("text").contains(word) for word in negative_words2])

In [65]:
df_with_sentiment2 = df_clean.withColumn(
    "sentiment",
    when(positive_condition2, "positive")
    .when(negative_condition2, "negative")
    .otherwise("neutral")
)

Py4JJavaError: An error occurred while calling o156.withColumn.
: java.lang.StackOverflowError
	at org.apache.spark.sql.classic.ColumnNodeToExpressionConverter.$anonfun$apply$4(columnNodeSupport.scala:86)
	at scala.collection.immutable.ArraySeq.map(ArraySeq.scala:75)
	at scala.collection.immutable.ArraySeq.map(ArraySeq.scala:35)
	at org.apache.spark.sql.classic.ColumnNodeToExpressionConverter.$anonfun$apply$2(columnNodeSupport.scala:86)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:86)
	at org.apache.spark.sql.classic.ColumnNodeToExpressionConverter.$anonfun$apply$1(columnNodeSupport.scala:48)
	at org.apache.spark.sql.internal.SQLConf$.withExistingConf(SQLConf.scala:161)
	at org.apache.spark.sql.classic.ColumnNodeToExpressionConverter.apply(columnNodeSupport.scala:47)
	at org.apache.spark.sql.classic.ColumnNodeToExpressionConverter.apply$(columnNodeSupport.scala:46)
	at org.apache.spark.sql.classic.SparkSession$$anon$3.apply(SparkSession.scala

#### TRYING IT IN CHUNKS

In [67]:
def chunk_conditions(words, chunk_size=100):
    chunks = [words[i:i+chunk_size] for i in range(0, len(words), chunk_size)]
    conditions = [reduce(lambda a, b: a | b, [col("text").contains(word) for word in chunk]) for chunk in chunks]
    return reduce(lambda a, b: a | b, conditions)

positive_condition2 = chunk_conditions(positive_words2)
negative_condition2 = chunk_conditions(negative_words2)

In [69]:
df_with_sentiment2 = df_clean.withColumn(
    "sentiment",
    when(positive_condition2, "positive")
    .when(negative_condition2, "negative")
    .otherwise("neutral")
)

In [71]:
df_with_sentiment2.show(5)

+--------------------+-------------------+-------------------+-------------------+--------------------+---------+
|                 _id|          author_id|         created_at|                 id|                text|sentiment|
+--------------------+-------------------+-------------------+-------------------+--------------------+---------+
|6891a2e83a8bc8d49...|1373906096792989696|2025-08-04 14:42:03|1952364410074234928|or could it be th...| positive|
|6891a2e83a8bc8d49...|1756438119287566336|2025-08-04 14:41:53|1952364370069217638|rt  elon i worked...| positive|
|6891a2e83a8bc8d49...|1154966639408013312|2025-08-04 14:42:06|1952364425240826229|in the us at leas...| negative|
|6891a2e83a8bc8d49...|         3389907262|2025-08-04 14:42:20|1952364480869826560|rt  disney planne...| negative|
|6891a2e83a8bc8d49...|1331232643330023429|2025-08-04 14:42:04|1952364414289764415|rt  breaking lock...| positive|
+--------------------+-------------------+-------------------+-------------------+------

In [73]:
df_with_sentiment2.printSchema()

root
 |-- _id: string (nullable = true)
 |-- author_id: long (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- sentiment: string (nullable = false)



### Exporting df_with_sentiment2 to PostgreSQL

In [75]:
sparkb = SparkSession.builder \
    .appName("PostgreSQL Integration") \
    .config("spark.jars", "C:\\Users\\ADURA\\Documents\\spark_jars\\postgresql-42.5.2.jar") \
    .getOrCreate()

#Testing connection by loading
df = sparkb.read.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/tweet_sentiments") \
    .option("dbtable", "tweets") \
    .option("user", "postgres") \
    .option("password", "*556#") \
    .load()

In [77]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- author_id: long (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [79]:
df.show()

+---+---------+----------+---+----+---------+
|_id|author_id|created_at| id|text|sentiment|
+---+---------+----------+---+----+---------+
+---+---------+----------+---+----+---------+



In [81]:
df_with_sentiment2.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://localhost:5432/tweet_sentiments") \
  .option("dbtable", "tweets") \
  .option("user", "postgres") \
  .option("password", "*556#") \
  .mode("append") \
  .save()

### End Spark Sessions

In [83]:
spark.stop()

In [85]:
sparkb.stop()

In [ ]:
import findspark
from pyspark.sql import SparkSession
#Functions for Preprocessing
from functools import reduce
from pyspark.sql.functions import lower, col, sum, when, regexp_replace, trim
from pyspark.sql.types import StringType, LongType, TimestampType

spark = SparkSession.builder \
    .appName("MongoDB Integration") \
    .config("spark.jars", "C:\\Users\\ADURA\\Documents\\spark_jars\\mongo-spark-connector_2.13-10.5.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\mongodb-driver-sync-4.9.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\mongodb-driver-core-4.9.0.jar,"
            "C:\\Users\\ADURA\\Documents\\spark_jars\\bson-4.9.0.jar").getOrCreate()

df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
    .option("spark.mongodb.read.database", "tweet_db") \
    .option("spark.mongodb.read.collection", "tweets") \
    .load()

#Text manipulation using regular expression (regex)
df = df.withColumn('text', regexp_replace(col('text'), r'http\S+', ''))  # Remove URLs
df = df.withColumn('text', regexp_replace(col('text'), r'@\w+', ''))     # Remove mentions
df = df.withColumn('text', regexp_replace(col('text'), r'#\w+', ''))     # Remove hashtags
df = df.withColumn('text', regexp_replace(col('text'), r'[^a-zA-Z\s]', ''))  # Remove special characters
df = df.withColumn('text', regexp_replace(col('text'), r'\n', ''))
df = df.withColumn('text', trim(col('text'))) #Remove leading or trailing white spaces